In [ ]:
#!pip install pyodbc
#!pip install pandas

In [ ]:
import pyodbc
import pandas as pd
import csv
import struct

In [ ]:
TABLES = [
    'AudioFile',
    'Practice', 
    'Menu',
]

In [ ]:
SERVER = 'localhost\SQLEXPRESS'
DATABASE = 'dev2'
UID = 'user01'
PWD = 'test123'

# DRIVER に指定できるODBCドライバ名のリストは、
# 下記のようにdriversメソッド実行で確認し、適当なものを指定。
# import pyodbc
# pyodbc.drivers()
con = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+UID+';PWD='+PWD)

# datetimeoffset型の対策
#  参考：https://stackoverflow.com/questions/46405373/odbc-sql-type-155-is-not-yet-supported
def handle_datetimeoffset(dto_value):
    # ref: https://github.com/mkleehammer/pyodbc/issues/134#issuecomment-281739794
    tup = struct.unpack("<6hI2h", dto_value)  # e.g., (2017, 3, 16, 10, 35, 18, 0, -6, 0)
    tweaked = [tup[i] // 100 if i == 6 else tup[i] for i in range(len(tup))]
    return "{:04d}-{:02d}-{:02d} {:02d}:{:02d}:{:02d}.{:07d} {:+03d}:{:02d}".format(*tweaked)
con.add_output_converter(-155, handle_datetimeoffset)

for t in TABLES:
    query = 'SELECT * FROM %s;' % (t)
    out_file = '.\_out\%s.csv' % (t)
    print('-'*100)
    print('query    : %s' % (query))
    print('out_file : %s' % (out_file))
    df = pd.read_sql(query, con)
    df.to_csv(out_file, index=False, encoding='utf-8', quoting=csv.QUOTE_NONNUMERIC)
con.close()